In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split

In [2]:
url = 'https://www.kaggle.com/datasets/d4rklucif3r/restaurant-reviews'

In [37]:
Review = []
Liked = []
with open('Restaurant_Reviews.tsv') as f:
  for line in f.readlines():
    r,l = line.strip().split('\t')
    Review.append(r)
    Liked.append(l)
df = pd.DataFrame({'Review':Review[1:], 'Liked':Liked[1:]})
df['Liked'] = df['Liked'].astype('int64')
df

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [38]:
x = df['Review']
y = df['Liked']
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)
train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
train = train.batch(64).prefetch(tf.data.AUTOTUNE)
test = test.batch(64).prefetch(tf.data.AUTOTUNE)

In [39]:
vectorizer = keras.layers.TextVectorization(max_tokens=1000)
vectorizer.adapt(train.map(lambda review, label: review))
model = keras.Sequential([
    vectorizer,
    keras.layers.Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=64, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [40]:
model.fit(train, epochs=32, validation_data=test)

Epoch 1/32
13/13 [==============================] - 15s 233ms/step - loss: 0.6921 - accuracy: 0.5275 - val_loss: 0.6899 - val_accuracy: 0.5750
Epoch 2/32
13/13 [==============================] - 0s 14ms/step - loss: 0.6768 - accuracy: 0.6662 - val_loss: 0.6619 - val_accuracy: 0.7000
Epoch 3/32
13/13 [==============================] - 0s 15ms/step - loss: 0.5781 - accuracy: 0.8087 - val_loss: 0.5105 - val_accuracy: 0.7900
Epoch 4/32
13/13 [==============================] - 0s 15ms/step - loss: 0.3564 - accuracy: 0.8825 - val_loss: 0.5223 - val_accuracy: 0.8000
Epoch 5/32
13/13 [==============================] - 0s 15ms/step - loss: 0.2358 - accuracy: 0.9250 - val_loss: 0.4845 - val_accuracy: 0.8050
Epoch 6/32
13/13 [==============================] - 0s 15ms/step - loss: 0.2442 - accuracy: 0.9237 - val_loss: 0.4888 - val_accuracy: 0.8300
Epoch 7/32
13/13 [==============================] - 0s 15ms/step - loss: 0.1668 - accuracy: 0.9425 - val_loss: 0.3899 - val_accuracy: 0.8400
Epoch 8/32


In [41]:
model.evaluate(test)

4/4 [==============================] - 0s 11ms/step - loss: 1.3435 - accuracy: 0.8150


[1.3434916734695435, 0.8149999976158142]